In [7]:
import random
import pandas as pd

In [8]:
mappings = {
  'San Francisco 49ers': '49ers',
  'Chicago Bears': 'Bears',
  'Cincinnati Bengals': 'Bengals',
  'Buffalo Bills': 'Bills',
  'Denver Broncos': 'Broncos',
  'Cleveland Browns': 'Browns',
  'Tampa Bay Buccaneers': 'Buccaneers',
  'Arizona Cardinals': 'Cardinals',
  'Los Angeles Chargers': 'Chargers',
  'Kansas City Chiefs': 'Chiefs',
  'Indianapolis Colts': 'Colts',
  'Washington Football Team': 'Commanders',
  'Washington Commanders': 'Commanders',
  'Dallas Cowboys': 'Cowboys',
  'Miami Dolphins': 'Dolphins',
  'Philadelphia Eagles': 'Eagles',
  'Atlanta Falcons': 'Falcons',
  'New York Giants': 'Giants',
  'Jacksonville Jaguars': 'Jaguars',
  'New York Jets': 'Jets',
  'Detroit Lions': 'Lions',
  'Green Bay Packers': 'Packers',
  'Carolina Panthers': 'Panthers',
  'New England Patriots': 'Patriots',
  'Las Vegas Raiders': 'Raiders',
  'Los Angeles Rams': 'Rams',
  'Baltimore Ravens': 'Ravens',
  'New Orleans Saints': 'Saints',
  'Seattle Seahawks': 'Seahawks', 
  'Pittsburgh Steelers': 'Steelers',
  'Houston Texans': 'Texans',
  'Tennessee Titans': 'Titans',
  'Minnesota Vikings': 'Vikings',
}

teams = list(set(mappings.values()))

In [18]:
def generate_team_power_ratings(random_range = (-15, 15)):
  power_rating = random.randint(random_range[0], random_range[1])
  power_rating += random.random()
  return { team: power_rating for team in teams }

home_power_ratings = generate_team_power_ratings()
away_power_ratings = generate_team_power_ratings()

In [16]:
df = pd.read_csv('../../data/nfl/pfr-2021-games-combo.csv', index_col=0)

df = df[df['week'].str.isdigit()]

df['week'] = df['week'].astype(str)
df['home'] = df['home'].astype(str).map(lambda a: mappings[a])
df['home_pts'] = df['home_pts'].astype(int)
df['away'] = df['away'].astype(str).map(lambda a: mappings[a])
df['away_pts'] = df['away_pts'].astype(int)
df['margin'] = df.apply(lambda a: a['home_pts'] - a['away_pts'], axis=1)

In [113]:
def tweak_power_ratings(power_ratings, random_range = (-1, 1), k_range = None):
   delta_power_ratings = power_ratings.copy()

   teams = list(delta_power_ratings.keys())

   if k_range is not None:
      k = random.randint(k_range[0], k_range[1])
   else:
      k = random.randint(2, len(teams))
   
   for team in random.sample(teams, k):
      delta_power_ratings[team] += random.randint(random_range[0], random_range[1])
      delta_power_ratings[team] += random.random()

   return delta_power_ratings

tweak_power_ratings(home_power_ratings)

{'Cardinals': 1.7635456797960343,
 'Texans': -7.0883432544860625,
 'Bengals': 2.1554063588898416,
 'Rams': 3.867736248059341,
 'Seahawks': 6.759613077827628,
 'Cowboys': 14.015215912700143,
 'Chargers': 6.437087049870834,
 'Steelers': -1.645984574151448,
 'Ravens': 8.09137638819605,
 'Vikings': 6.0822796850110885,
 'Lions': -3.0625288139273863,
 'Saints': -4.6221485598297285,
 'Bills': 7.4373570557455935,
 'Jaguars': -9.96330994227373,
 'Dolphins': -1.2104734599213205,
 'Raiders': -4.6800190011307965,
 'Jets': -11.370808291267027,
 'Commanders': -5.504908272948102,
 'Patriots': 4.670011763744691,
 'Falcons': -8.023375257599092,
 'Titans': 8.025192922055211,
 'Browns': -1.505533946073749,
 'Giants': -5.77435675170853,
 'Panthers': -9.877864520410267,
 'Buccaneers': 20.69758444521811,
 'Chiefs': 11.234903049005803,
 '49ers': 4.729781221371203,
 'Eagles': -0.6976201924051977,
 'Packers': 11.179412329987349,
 'Broncos': 8.56849464754345,
 'Colts': -1.7787569654972892,
 'Bears': 1.032993398

In [78]:
def evaluate(df, home_power_ratings, away_power_ratings, edge = 3):
  def prediction(row):
    return row['home_rating_adj'] - row['away_rating_adj']

  df_pred = df.copy()

  home_edge = .5 * edge
  away_edge = -home_edge

  df_pred['home_rating'] = df_pred['home'].map(home_power_ratings)
  df_pred['home_rating_adj'] = home_edge + df_pred['home_rating']
  df_pred['away_rating'] = df_pred['away'].map(away_power_ratings)
  df_pred['away_rating_adj'] = -away_edge + df_pred['away_rating']
  df_pred['prediction'] = df_pred.apply(prediction, axis=1)
  df_pred['error'] = df_pred.apply(lambda a: a['margin'] - a['prediction'], axis=1)
  df_pred['sq_error'] = df_pred['error'].map(lambda a: a * a)
  df_pred['abs_error'] = df_pred['error'].map(lambda a: abs(a))

  columns = [
    'week',
    'home',
    'home_pts',
    'home_rating',
    'home_rating_adj',
    'away',
    'away_pts',
    'away_rating',
    'away_rating_adj',
    'prediction',
    'error',
    'sq_error',
    'abs_error',
  ]

  return df_pred[columns]

df_pred = evaluate(df, home_power_ratings, away_power_ratings)

print('sq error:', df_pred['sq_error'].sum())
print('abs error:', df_pred['abs_error'].sum())
print()

df_pred.head()

sq error: 48223.067281039264



,week,home,home_pts,home_rating,home_rating_adj,away,away_pts,away_rating,away_rating_adj,prediction,error,sq_error,abs_error
0,1,Buccaneers,31,16.004210,17.504210,Cowboys,29,10.206490,11.706490,5.797721,-3.797721,14.422684,3.797721
1,1,Falcons,6,-6.550256,-5.050256,Eagles,32,1.691250,3.191250,-8.241506,-17.758494,315.364115,17.758494
2,1,Bills,16,16.900394,18.400394,Steelers,23,2.123089,3.623089,14.777305,-21.777305,474.251012,21.777305
3,1,Panthers,19,-4.836563,-3.336563,Jets,14,-12.511374,-11.011374,7.674811,-2.674811,7.154612,2.674811
4,1,Bengals,27,-4.817072,-3.317072,Vikings,24,-2.899676,-1.399676,-1.917396,4.917396,24.180780,4.917396


In [79]:
home_power_ratings = generate_team_power_ratings()
away_power_ratings = generate_team_power_ratings()

error_col = 'abs_error'

df_eval = evaluate(df, home_power_ratings, away_power_ratings)
global_error = df_eval[error_col].sum()

In [111]:
passes = 10
total_iterations_allowed_in_pass = 500
random_range = (-3, 3)
k_range = (2, 8)

print(f'global error @ 0:', global_error)

for i in range(1, passes):

  tse = global_error

  current_iteration = 1
  while current_iteration <= total_iterations_allowed_in_pass:
    delta_home_power_ratings = tweak_power_ratings(home_power_ratings, random_range, k_range)
    df_eval = evaluate(df, delta_home_power_ratings, away_power_ratings)
    error = df_eval[error_col].sum()
    if global_error > error:
      global_error = error
      home_power_ratings = delta_home_power_ratings

    delta_away_power_ratings = tweak_power_ratings(away_power_ratings, random_range, k_range)
    df_eval = evaluate(df, home_power_ratings, delta_away_power_ratings)
    error = df_eval[error_col].sum()
    if global_error > error:
      global_error = error
      away_power_ratings = delta_away_power_ratings

    current_iteration += 1

  if tse != global_error:
    print(f'global error @ {i}:', global_error)
  


global error @ 0: 2374.1354287954155
global error @ 2: 2373.687522359676
global error @ 3: 2373.4873999391802
global error @ 5: 2373.439571627924
global error @ 6: 2373.4326370998633
global error @ 7: 2373.378086783806
global error @ 8: 2373.2943034898435


In [107]:
##

In [108]:
print('sq error:', global_error)
evaluate(df, home_power_ratings, away_power_ratings).head()

sq error: 2374.1354287954155


,week,home,home_pts,home_rating,home_rating_adj,away,away_pts,away_rating,away_rating_adj,prediction,error,sq_error,abs_error
0,1,Buccaneers,31,19.422373,20.922373,Cowboys,29,9.846685,11.346685,9.575689,-7.575689,57.391058,7.575689
1,1,Falcons,6,-8.251169,-6.751169,Eagles,32,-1.494762,0.005238,-6.756407,-19.243593,370.315862,19.243593
2,1,Bills,16,7.437357,8.937357,Steelers,23,2.012037,3.512037,5.425320,-12.425320,154.388581,12.425320
3,1,Panthers,19,-9.568092,-8.068092,Jets,14,-14.729874,-13.229874,5.161782,-0.161782,0.026174,0.161782
4,1,Bengals,27,2.155406,3.655406,Vikings,24,-0.709170,0.790830,2.864577,0.135423,0.018339,0.135423


In [109]:
sorted(home_power_ratings.items(), key=lambda a: a[1], reverse=True)

[('Buccaneers', 19.422373366258046),
 ('Cowboys', 14.937468650626114),
 ('Packers', 11.971832783523968),
 ('Chiefs', 9.808836313463582),
 ('Titans', 8.645819147262578),
 ('Ravens', 8.09137638819605),
 ('Bills', 7.4373570557455935),
 ('Vikings', 7.293517367872055),
 ('Broncos', 6.78883146441204),
 ('Seahawks', 6.759613077827628),
 ('Chargers', 6.437087049870834),
 ('Patriots', 4.617021448439424),
 ('49ers', 4.596697100237252),
 ('Rams', 3.867736248059341),
 ('Bears', 2.9328990892686373),
 ('Bengals', 2.1554063588898416),
 ('Cardinals', 1.8191295912567),
 ('Eagles', -0.4659597894996562),
 ('Browns', -0.6284384784835987),
 ('Colts', -1.053787297131895),
 ('Dolphins', -1.2104734599213205),
 ('Steelers', -1.645984574151448),
 ('Lions', -2.5354830553512073),
 ('Saints', -4.256965326652278),
 ('Raiders', -4.6800190011307965),
 ('Giants', -5.272256375051459),
 ('Commanders', -5.514016339848246),
 ('Texans', -7.0883432544860625),
 ('Falcons', -8.251169040203107),
 ('Panthers', -9.56809178476391

In [110]:
sorted(away_power_ratings.items(), key=lambda a: a[1], reverse=True)

[('Cardinals', 17.92471610486971),
 ('Bills', 16.680515436763255),
 ('Bengals', 11.810117365167352),
 ('Chiefs', 11.050841670407134),
 ('Cowboys', 9.84668472651552),
 ('Rams', 8.920961395306028),
 ('Colts', 8.792893731373287),
 ('Patriots', 8.341220650363017),
 ('Saints', 6.513139881175496),
 ('49ers', 5.492148563519303),
 ('Browns', 5.413925912493311),
 ('Buccaneers', 5.3074878551788265),
 ('Packers', 4.810053835798759),
 ('Steelers', 2.0120368930462496),
 ('Raiders', 1.4065299046616657),
 ('Titans', -0.3566109768840592),
 ('Vikings', -0.7091703409980411),
 ('Eagles', -1.4947617983564085),
 ('Chargers', -2.0827763578065035),
 ('Ravens', -2.641013533478342),
 ('Falcons', -2.9740952405420855),
 ('Commanders', -3.210554641112485),
 ('Broncos', -3.3239423901072094),
 ('Bears', -3.4638099199673),
 ('Panthers', -4.845157413375357),
 ('Seahawks', -4.924472656100296),
 ('Lions', -6.0975252221369685),
 ('Dolphins', -7.674845214554322),
 ('Giants', -9.035324461973037),
 ('Texans', -11.401501687